# Zadanie 5

Celem ćwiczenia jest implementacja algorytmu Q-learning.

Następnie należy stworzyć agenta rozwiązującego problem [Taxi](https://gymnasium.farama.org/environments/toy_text/taxi/). Problem dostępny jest w pakiecie **gym**.

Punktacja (max 7 pkt):
- Implementacja algorytmu Q-learning. [3 pkt]
- Eksperymenty dla różnych wartości hiperparametrów [2 pkt]
- Jakość kodu [1 pkt]
- Wnioski [1 pkt]


In [16]:
import numpy as np
import gymnasium as gym

In [17]:
class QLearningSolver:
    """Class containing the Q-learning algorithm that might be used for different discrete environments."""

    def __init__(
        self,
        observation_space: int,
        action_space: int,
        learning_rate: float = 0.9,
        gamma: float = 0.9,
        epsilon: float = 0.1,
        q_table: np.ndarray = None,
    ):
        self.observation_space = observation_space
        self.action_space = action_space
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.epsilon = epsilon
        if q_table is None:
            self.q_table = np.zeros(shape=(observation_space, action_space))
        else:
            self.q_table = q_table

    def __call__(self, state: np.ndarray, action: np.ndarray) -> float:
        """Return Q-value of given state and action."""
        return self.q_table[state][action]

    def update(self, state: np.ndarray, action: np.ndarray, reward: float) -> None:
        """Update Q-value of given state and action."""
        self.q_table[state][action] += reward

    def get_best_action(self, state: np.ndarray) -> int:
        """Return action that maximizes Q-value for a given state."""
        return np.argmax(self.q_table[state])

    def get_best_move_evaluation(self, state: np.array) -> float:
        return np.max(self.q_table[state])

    def __repr__(self):
        """Elegant representation of Q-learning solver."""
        pass

    def __str__(self):
        return self.__repr__()

In [39]:
def run_episode(solver: QLearningSolver, environment):
    state = environment.reset()[0]
    terminated, truncated = False, False

    while not terminated and not truncated:
        if np.random.random() < solver.epsilon:
            action = environment.action_space.sample()
        else:
            action = solver.get_best_action(state)

        next_state, reward, terminated, truncated, _ = environment.step(action)
        delta = (
            reward
            + solver.gamma * solver.get_best_move_evaluation(next_state)
            - solver(state, action)
        )
        solver.update(state, action, solver.learning_rate * delta)
        state = next_state


def q_learning(
    environment, learning_rate=0.9, gamma=0.9, epsilon=0.1, number_of_episodes=1000
):
    solver = QLearningSolver(
        environment.observation_space.n,
        environment.action_space.n,
        learning_rate,
        gamma,
        epsilon,
    )
    for _ in range(number_of_episodes):
        run_episode(solver, environment)
    return solver

In [40]:
def test_solver(solver: QLearningSolver, environment, number_of_tests: int = 100):
    successes = 0
    total_steps = 0
    for _ in range(number_of_tests):
        state = environment.reset()[0]
        terminated, truncated = False, False
        steps = 0
        while not terminated and not truncated:
            action = solver.get_best_action(state)
            next_state, reward, terminated, truncated = environment.step(action)[:4]
            state = next_state
            steps += 1
        if terminated and reward > 0:
            successes += 1
        total_steps += steps
    return successes / number_of_tests, total_steps / number_of_tests

In [86]:
def run_experiment(
    env, learning_rate=0.9, gamma=0.9, epsilon=0.1, number_of_episodes=1000
):
    solver = q_learning(env, learning_rate, gamma, epsilon, number_of_episodes)
    success_rate, average_number_of_steps = test_solver(solver, env)
    print("LEARNING:")
    print(
        f"Learning rate: {learning_rate}, gamma: {gamma}, epsilon: {epsilon}, number of episodes: {number_of_episodes}"
    )
    print("TESTING:")
    print(
        f"Success Rate: {success_rate}\nAverage number of steps: {average_number_of_steps}"
    )

# Testy

In [51]:
env = gym.make("Taxi-v3")

In [70]:
run_experiment(env)

LEARNING:
Learning rate: 0.9, gamma: 0.9, epsilon: 0.1, number of episodes: 1000
TESTING:
Success Rate: 0.98
Averagenumber of steps: 16.69


In [71]:
run_experiment(env, number_of_episodes=800)

LEARNING:
Learning rate: 0.9, gamma: 0.9, epsilon: 0.1, number of episodes: 800
TESTING:
Success Rate: 0.94
Averagenumber of steps: 24.62


In [72]:
run_experiment(env, number_of_episodes=600)

LEARNING:
Learning rate: 0.9, gamma: 0.9, epsilon: 0.1, number of episodes: 600
TESTING:
Success Rate: 0.9
Averagenumber of steps: 32.11


In [117]:
run_experiment(env, number_of_episodes=1500)

LEARNING:
Learning rate: 0.9, gamma: 0.9, epsilon: 0.1, number of episodes: 1500
TESTING:
Success Rate: 1.0
Average number of steps: 13.01


In [118]:
run_experiment(env, gamma = 0.7)

LEARNING:
Learning rate: 0.9, gamma: 0.7, epsilon: 0.1, number of episodes: 1000
TESTING:
Success Rate: 0.97
Average number of steps: 18.46


In [119]:
run_experiment(env, gamma = 0.7, number_of_episodes=1500)

LEARNING:
Learning rate: 0.9, gamma: 0.7, epsilon: 0.1, number of episodes: 1500
TESTING:
Success Rate: 1.0
Average number of steps: 13.17


In [120]:
run_experiment(env, learning_rate=0.6)

LEARNING:
Learning rate: 0.6, gamma: 0.9, epsilon: 0.1, number of episodes: 1000
TESTING:
Success Rate: 0.98
Average number of steps: 17.0


In [80]:
run_experiment(env, learning_rate=0.8, epsilon=0.5)

LEARNING:
Learning rate: 0.8, gamma: 0.9, epsilon: 0.5, number of episodes: 1000
TESTING:
Success Rate: 1.0
Averagenumber of steps: 12.99


In [84]:
run_experiment(env, learning_rate=0.5, epsilon=0.5)

LEARNING:
Learning rate: 0.5, gamma: 0.9, epsilon: 0.5, number of episodes: 1000
TESTING:
Success Rate: 1.0
Averagenumber of steps: 12.96


In [85]:
run_experiment(env, learning_rate=0.1, epsilon=0.5)

LEARNING:
Learning rate: 0.1, gamma: 0.9, epsilon: 0.5, number of episodes: 1000
TESTING:
Success Rate: 0.44
Averagenumber of steps: 116.77


In [27]:
run_experiment(env, learning_rate=0.9, epsilon=0.5)

Success ratio: 0.853
Average number of steps: 18.173


In [104]:
run_experiment(env, learning_rate=0.9, gamma=0.9, epsilon=0.25)

LEARNING:
Learning rate: 0.9, gamma: 0.9, epsilon: 0.25, number of episodes: 1000
TESTING:
Success Rate: 0.99
Average number of steps: 15.41


In [108]:
run_experiment(env, learning_rate=0.2, gamma=0.9, epsilon=0.4, number_of_episodes=2500)

LEARNING:
Learning rate: 0.2, gamma: 0.9, epsilon: 0.4, number of episodes: 2500
TESTING:
Success Rate: 1.0
Average number of steps: 12.94


In [109]:
run_experiment(env, learning_rate=0.2, gamma=0.9, epsilon=0.4)

LEARNING:
Learning rate: 0.2, gamma: 0.9, epsilon: 0.4, number of episodes: 1000
TESTING:
Success Rate: 0.72
Average number of steps: 65.29


In [113]:
run_experiment(env, learning_rate=0.8, gamma=0.4, epsilon=0.4)

LEARNING:
Learning rate: 0.8, gamma: 0.4, epsilon: 0.4, number of episodes: 1000
TESTING:
Success Rate: 0.9
Average number of steps: 31.9


In [115]:
run_experiment(env, learning_rate=0.8, gamma=0.4, epsilon=0.7)

LEARNING:
Learning rate: 0.8, gamma: 0.4, epsilon: 0.7, number of episodes: 1000
TESTING:
Success Rate: 0.99
Average number of steps: 15.16


In [114]:
run_experiment(env, learning_rate=0.9, gamma=0.4, epsilon=0.4)

LEARNING:
Learning rate: 0.9, gamma: 0.4, epsilon: 0.4, number of episodes: 1000
TESTING:
Success Rate: 0.96
Average number of steps: 20.88


In [146]:
run_experiment(env, learning_rate=0.99, gamma=0.8, epsilon=0.2, number_of_episodes=1000)

LEARNING:
Learning rate: 0.99, gamma: 0.8, epsilon: 0.2, number of episodes: 1000
TESTING:
Success Rate: 0.95
Average number of steps: 22.16


In [156]:
run_experiment(env, learning_rate=0.7, gamma=0.8, epsilon=0.2, number_of_episodes=1000)

LEARNING:
Learning rate: 0.7, gamma: 0.8, epsilon: 0.2, number of episodes: 1000
TESTING:
Success Rate: 0.96
Average number of steps: 20.57


In [121]:

solver = q_learning(
    env, learning_rate=0.9, gamma=0.9, epsilon=0.4, number_of_episodes=1000
)
np.save("solver", solver.q_table)

# Wnioski

Skuteczność algorytmu QLearning w dużym stopniu zależy od poziomu skomplikowania środowisk i dopowiedniego doboru hiperparametrów. 

Jeśli epsilon jest zbyt mały, przestrzeń nie będzie eksplorowana w odpowiednim stopniu, jednak jeśli epsilon będzie zbyt duży, zdobyta wiedza będzie wykorzystywana w małym stopniu, a co za tym idzie algorytm może nie dojść do stanu akceptującego w rozsądnym czasie.

Zbyt duży learning rate powoduje, że to czego algorytm nauczy się w początkowych epizodach może zostać zapomniane w trakcie późniejszych epizodów, co poskutkuje obniżeniem skuteczności. Jednak mniejszy learning rate, sprawia że potrzeba większej liczby epizodów trenujących do uzyskania satysfakcjonujących wyników. 

Gamma jest odpowiedzialna za szybkość dążenia do potencjalnych nagród. Duża wartość parametru, wskazuje że preferowane są większe nagrody, nawet jeśli do ich uzysania należy poświęcić więcej wysiłku (preferencja nagród długoterminowych). Zbyt duża wartość gammy prowadzi do małej eksploatacji, natomiast zbyt mała gamma prowadzi do za małej eksploracji.

W skomplikowanych środowiskach pomocne okazać się może zmniejszanie hiperparametrów epsilon i learning rate wraz ze wzrostem wiedzy na temat środowiska, a także zmniejszanie parametru gamma w późniejszych fazach każdego epizodu. Zmniejszanie epsilonu pozwala na wykorzystanie zdobytej wiedzy, dzięki czemu algorytm nie musi się uczyć kilka razy tych samych ścieżek. Dzięki zmniejszeniu learning rate algorytm nie zapomina informacji, które zdobył w trakcie wcześniejszych epizodów. Stopniowe zmniejszanie parametru gamma w każdym epizodzie może pozwolić na preferowanie szybkich nagród, w późniejszych fragmentach epizodu, czyli kiedy algorytm nie ma dużo czasu na próby zdobycia nagród długoterminowych.